# Visualisation of the envelope in borsuk

### Importing necessary libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import rcParams
import os.path
import statsmodels.api as sm
import statistics
import http.client
import json
import urllib.request
import requests
import random
from urllib import error
import configparser 
import time
import datetime
import re

### Accessing the token
Pobranie danych dla kanału

In [2]:
config = configparser.ConfigParser()
config.read("config.txt")
http_client = config.get("configuration","http_client")
conn = http.client.HTTPConnection(http_client)
headers = {'Content-type': 'application/json'}
Username = config.get("configuration","user")
Password = config.get("configuration","password")
body = { 'Username': Username, 'Password': Password}
conn.request("POST","/auth/authenticate", json.dumps(body),headers)
response = conn.getresponse()
x = response.read().decode()
token = json.loads(x)['token']

### Loading the data set

In [3]:
url_rain = 'http://13.77.168.238/data/channel/3231-3048/data?token=' + token
url_flow = 'http://13.77.168.238/data/channel/1928-1363/data?token=' + token
df_rain = pd.read_csv(url_rain, parse_dates=['time'])
df_flow = pd.read_csv(url_flow, parse_dates=['time'])

### Filling missing dates in datasets

In [4]:
df_rain = df_rain.set_index('time')
df_rain = df_rain.resample('300S').asfreq()
df_flow = df_flow.set_index('time')
df_flow = df_flow.resample('900S').asfreq()

In [5]:
df_rain = df_rain.reset_index()

In [6]:
df_flow = df_flow.reset_index()

In [7]:
df_rain = df_rain.fillna(-1)
df_flow = df_flow.fillna(-1)

### Taking the fragment of the data to the model

In [8]:
split_rain = df_rain[(df_rain['time'] > '2012-09-16 00:00:00')]
split_rain = split_rain[(split_rain['time'] < '2016-4-9 00:00:00')]
split_flow = df_flow[(df_flow['time'] > '2012-09-16 00:00:00')]
split_flow = split_flow[(split_flow['time'] < '2016-4-9 00:00:00')]

### Creating json variable

In [9]:
""" {
	"rainfall" : {
		"start-date" : "2012-09-16 00:05:00",
		"resolution" : "PT5M",
		"values" : [1.0,2.0,3.0]
	},
	"flow" : {
		"start-date" : "2012-09-16 00:15:00",
		"resolution" : "PT5M",
		"values" : [2.0,3.0,4.0]
	},
	"stormIntensityWindow" : "PT6H",
	"flowIntensityWindow" : "PT1H",
	"dryDayWindow" : "P2D",
	"minSessionWindow" : "PT12H",
	"maxSessionWindow" : "PT12H"
} """

' {\n\t"rainfall" : {\n\t\t"start-date" : "2012-09-16 00:05:00",\n\t\t"resolution" : "PT5M",\n\t\t"values" : [1.0,2.0,3.0]\n\t},\n\t"flow" : {\n\t\t"start-date" : "2012-09-16 00:15:00",\n\t\t"resolution" : "PT5M",\n\t\t"values" : [2.0,3.0,4.0]\n\t},\n\t"stormIntensityWindow" : "PT6H",\n\t"flowIntensityWindow" : "PT1H",\n\t"dryDayWindow" : "P2D",\n\t"minSessionWindow" : "PT12H",\n\t"maxSessionWindow" : "PT12H"\n} '

In [10]:
rain_json = split_rain.to_json(orient='split')
flow_json = split_flow.to_json(orient='split')
rain_values = split_rain['value']
rain_values = np.array(rain_values)
rain_values = rain_values.tolist()
flow_values = split_flow['value']
flow_values = np.array(flow_values)
flow_values = flow_values.tolist()

In [11]:
envelope =  {
	"rainfall" : {
		"start-date" : "2018-01-01T00:00:00",
		"resolution" : "PT5M",
		"values" : []
	},
	"flow" : {
		"start-date" : "2018-01-01T00:00:00",
		"resolution" : "PT15M",
		"values" : []
	},
	"stormIntensityWindow" : "PT6H",
	"flowIntensityWindow" : "PT1H",
	"dryDayWindow" : "P2D",
	"minSessionWindow" : "PT12H",
	"maxSessionWindow" : "PT12H"
}

In [12]:
envelope['flow']['values'].append(flow_values)
envelope['rainfall']['values'].append(rain_values)

In [13]:
envelope = json.dumps(envelope)
envelope = envelope.replace('[[','[')
envelope = envelope.replace(']]',']')
envelope = json.loads(envelope)

### POST to borsuk-envelope
{"type":"envelope-v0", "id": "model-id"}

In [14]:
env_url = config.get("configuration", "env_url")
body={"type":"envelope-v0","id": "env_test"}
response = requests.post(env_url, json=body)
print(response.status_code)
print(response.content) 

409
b'Error: Model with this id already exist.'


### GET to borsuk-envelope env-test

In [15]:
env_model = config.get("configuration", "env_model")
r = requests.get(env_model)
first = json.loads(r.content)['build']
print(first)

1


### POST to borsuk-envelope json
The change of status informs that the post is finished

In [16]:
env_fit = config.get("configuration", "env_fit")
response = requests.post(env_fit, json=envelope)
print(response.status_code)
print(response.content)

200
b'OK'


In [ ]:
print("Primary status: ",first)
start_time = time.time()

while True:
    r = requests.get(env_model)
    last = json.loads(r.content)['build']
    if last > first:
        print("Status after post: ",last)
        break

print("--- %s seconds ---" % (time.time() - start_time))

### GET to borsuk-envelope to list envelopes

In [1]:
''' env_list = config.get("configuration", "env_id")
r = requests.get(env_list)
b = json.loads(r.content) '''

' env_list = config.get("configuration", "env_id")\nr = requests.get(env_list)\nb = json.loads(r.content) '